[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fw-ai/cookbook/blob/main/examples/function_calling/fireworks_functions_information_extraction.ipynb)

# Summarize Anything - Information Extraction via [Fireworks Function Calling](https://readme.fireworks.ai/docs/function-calling)

This is inspired by awesome colab notebook by [Deepset](https://colab.research.google.com/github/anakin87/notebooks/blob/main/information_extraction_via_llms.ipynb). Checkout there OSS LLM Orchestration framework [haystack](https://haystack.deepset.ai/).

In this experiment, we will use function calling ability of [Fireworks Function Calling](https://readme.fireworks.ai/docs/function-calling) model to generate structured information from unstrucutred data.

🎯 Goal: create an application that, given a text (or URL) and a specific structure provided by the user, extracts information from the source.


The "**function calling**" capability first launched by [OpenAI](https://platform.openai.com/docs/guides/function-calling) unlocks this task: the user can describe a structure, by defining a fake function with all its typed and specific parameters. The LLM will prepare the data in this specific form and send it back to the user.

**Fireworks Function Calling**

Fireworks released a high quality function calling model which is capable of handling long tool context, multi turn conversations & interleaving tool invocations ith regular conversation. We are going to use this model today as our LLM to power our app.



>[Summarize Anything - Information Extraction via Fireworks Function Calling](#scrollTo=8Ksv005GbN2w)

>>[Introduction](#scrollTo=cp4hJ34JivkB)

>[Document Retrieval & Clean Up](#scrollTo=buM6rGqMwLZ4)

>>>[Let's learn about Capybara](#scrollTo=0kVJ8IfSI-Dx)

>>>[How about Yucatan Deer](#scrollTo=Tzz1LSS-JBk4)

>>>[Something more massive - African Elephant](#scrollTo=B0M4NEm9JMAw)

>>[Let's make example fun - News Summarization](#scrollTo=x7Y8_xmxDOKx)



In [ ]:
!pip install openai

In [ ]:
import torch
import json
from typing import Dict
import os
import openai
from IPython.display import HTML, display

In [ ]:
model_name = "accounts/fireworks/models/fw-function-call-34b-v0"
clinet = client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = os.environ["FW_API_KEY"],
)

## Introduction

The [documentation](https://readme.fireworks.ai/docs/function-calling) for FW function calling details the API we can use to specify the list of tools/functions available to the model. We will use the described API to test out the structured response usecase.

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "uber.ride",
      "description": "Find suitable ride for customers given the location, type of ride, and the amount of time the customer is willing to wait as parameters",
      "parameters": {
          "type": "object",
          "properties": {
             "loc":  { "type": "string", "description": "location of the starting place of the uber ride"},
             "type": { "type": "enum", "enum": ["plus", "comfort", "black"], "description": "types of uber ride user is ordering"},
             "time": { "type": "string", "description": "the amount of time in minutes the customer is willing to wait"}
          }
      }
    }
  }
]
tool_choice = "auto"
user_prompt = "Call me an Uber ride type \"Plus\" in Berkeley at zipcode 94704 in 10 minute"
messages = [
    {
         "role": "system",
         "content": "You are a helpful assistant with access to tools. Use them wisely and don't image parameter values",
    },
    {
        "role": "user",
        "content": user_prompt,
    }
]

In [ ]:
chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [ ]:
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": " ",
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_m3SQZptP33ehkhux6ogd7zLC",
            "function": {
                "arguments": "{\"loc\": \"Berkeley\", \"type\": \"plus\", \"time\": \"10\"}",
                "name": "uber.ride"
            },
            "type": "function",
            "index": 0
        }
    ]
}


All good! ✅

# Document Retrieval & Clean Up

Before we can get started with extracting the right set of information. We need to first obtaint the document given a url & then clean it up. For cleaning up HTML, we will use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/).

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.rainforest-alliance.org/species/capybara/"
page = requests.get(url)

# Import Module
from bs4 import BeautifulSoup
import requests

# Website URL
URL = 'https://www.geeksforgeeks.org/data-structures/'

# Page content from Website URL
page = requests.get(url)

# Function to remove tags
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


# Print the extracted data
cleaned_content = remove_tags(page.content)

In [ ]:
from typing import Dict, List, Any

def extract_data(tools: List[Dict[str, Any]], url: str) -> str:
  assert len(tools) == 1, "Only one scehma can be selected for data extraction"
  tool_choice = {
      "type": "function",
      "function": {
          "name": tools[0]["function"]["name"]
      }
  }
  page = requests.get(url)
  cleaned_content = remove_tags(page.content)

  messages = [
      {
          "role": "system",
          "content": f"You are a helpful assistant with access to tools. Use them wisely and don't imageine parameter values."
      },
      {
          "role": "user",
          "content": f"Extract data from the following text. START TEXT {cleaned_content} END TEXT."
      }
  ]

  chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.0
  )

  def val_to_color(val):
    """
    Helper function to return a color based on the type/value of a variable
    """
    if isinstance(val, list):
      return "#FFFEE0"
    if val is True:
      return "#90EE90"
    if val is False:
      return "#FFCCCB"
    return ""

  args = json.loads(chat_completion.choices[0].message.tool_calls[0].function.arguments)

  # Convert data to HTML format
  html_content = '<div style="border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #f9f9f9;">'
  for key, value in args.items():
      html_content += f'<p><span style="font-family: Cursive; font-size: 30px;">{key}:</span>'
      html_content += f'&emsp;<span style="background-color:{val_to_color(value)}; font-family: Cursive; font-size: 20px;">{value}</span></p>'
  html_content += '</div>'

  return {"html_visualization": html_content}

In [ ]:
animal_info_tools = [
{
  "type": "function",
  "function": {
      "name": "extract_data",
      "description": "Extract data from text",
      "parameters": {
          "type": "object",
          "properties": {
              "about_animals": {
                  "description": "Is the article about animals?",
                  "type": "boolean",
              },
              "about_ai": {
                  "description": "Is the article about artificial intelligence?",
                  "type": "boolean",
              },
              "weight": {
                  "description": "the weight of the animal in lbs",
                  "type": "integer",
              },
              "habitat": {
                  "description": "List of places where the animal lives",
                  "type": "array",
                  "items": {"type": "string"},
              },
              "diet": {
                  "description": "What does the animal eat?",
                  "type": "array",
                  "items": {"type": "string"},
              },
              "predators": {
                  "description": "What are the animals that threaten them?",
                  "type": "array",
                  "items": {"type": "string"},
              },
          },
          "required": ["about_animals", "about_ai", "weight", "habitat", "diet", "predators"],
      }
  }
}
]

### Let's learn about Capybara

In [ ]:
display(HTML(extract_data(animal_info_tools, url="https://www.rainforest-alliance.org/species/capybara/")['html_visualization']))

### How about Yucatan Deer

In [ ]:
display(HTML(extract_data(animal_info_tools, url="https://www.rainforest-alliance.org/species/yucatan-deer/")['html_visualization']))

### Something more massive - African Elephant

In [ ]:
display(HTML(extract_data(animal_info_tools, url="https://www.rainforest-alliance.org/species/african-elephants/")['html_visualization']))

## Let's make example fun - News Summarization

Now let's use a more fun example. In order for LLMs to leverage the world knowledge, they need to be able to organize unstructured sources like websites into more structured information. Let's take the example of a news article announcing the new funding round for the startup [Perplexity AI](https://www.perplexity.ai/). For our sample news summarization app, the user only specifies the small list of information that want from the article and then ask the LLM to generate the needed information for them.

In [ ]:
# Detail the information needed
news_info_tools = [
  {
    "type": "function",
    "function": {
        "name": "extract_data",
        "description": "Extract data from text",
        "parameters": {
            "type": "object",
            "properties": {
                "about_ai": {
                    "description": "Is the article about artificial intelligence?",
                    "type": "boolean",
                },
                "company_name": {
                    "description": "The name of the company which is being referenced in document",
                    "type": "string",
                },
                "valuation": {
                    "description": "Valuation of the company which is being referenced in document",
                    "type": "string",
                },
                "investors": {
                    "description": "investors in the company being referenced in document",
                    "type": "array",
                    "items": {"type": "string"},
                },
                "competitors": {
                    "description": "competitors of the company being referenced in document",
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["about_ai", "company_name", "valuation", "investors", "competitors"],
        }
    }
  }
]

In [ ]:
display(HTML(extract_data(news_info_tools, url="https://finance.yahoo.com/news/perplexity-ai-challenge-google-hinges-124622631.html")['html_visualization']))